In [46]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings("ignore")

### Importation de la base de données

In [50]:
df=pd.read_csv("dataset/cleaned_datasetclusters.csv")

In [51]:
df.drop('Unnamed: 0', axis=1, inplace=True)

Ici nous nous interressons à la prédiction de montant total facturé TotalCharge et nous incluons pas la vaiable Churn dans le modèle de regression car elle représente notre variable dépendante pour le problème de Classification. Plusieurs modeles seront testés et optimisés et comparés entre eux pour choisir le modèle le plus optimal selon les métriques qu'on définira dans la suite

In [52]:
X = df.drop(["TotalCharges","Churn"], axis=1)
y = df["TotalCharges"]

On coupe le jeu de données en jeu de d'apprentissage et test sur la base de 80/20

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### **Knn**

Nous commencons par un modele de regression KNN simple sans optimisation des hyperparametres. L'erreur quadratique moyenne et le coefficient de détermination sont utilisés pour évaluer la qualité prédictive du modèle

#### Modèle naif sans optimisation

In [23]:
cols_to_scale = ["tenure", "MonthlyCharges"]
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[cols_to_scale] = scaler.fit_transform(X_train[cols_to_scale])
X_test_scaled[cols_to_scale] = scaler.transform(X_test[cols_to_scale])

In [32]:
knn = KNeighborsRegressor(n_neighbors=4)
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse_knn = np.sqrt(mse)
r2_knn = r2_score(y_test, y_pred)
print("RMSE :", rmse_knn, "R2 :", r2_knn)

RMSE : 381.22619423722165 R2 : 0.9720642217330197


#### Recherche d'hyperparamètre

A présent, nous allons essayer d'améliorer le modele naif précedent en allant à la recherche des hyperparamètres tout en contrôlant le surapprentissage par la validation croisée afin de garantir la robustesse des résultats.

In [36]:
knn = KNeighborsRegressor()
# Grille d'hyperparamètres 
param_grid = {"n_neighbors": list(range(3, 31)),"weights": ["uniform", "distance"],"p": [1, 2]}
# GridSearchCV avec validation croisée
grid_knn = GridSearchCV(knn,param_grid,cv=10,scoring="neg_mean_squared_error",n_jobs=-1)
# Entraînement du modèle
grid_knn.fit(X_train_scaled, y_train)
# Meilleurs paramètres 
best_params_knn = grid_knn.best_params_
print("Meilleurs paramètres KNN :", best_params_knn)

Meilleurs paramètres KNN : {'n_neighbors': 15, 'p': 2, 'weights': 'distance'}


Avec une validation croisée de 10 fold, la recherche d'hyperparamètre a identifié que le modele KNN est perfomant en utilisant 15 voisins, une distance euclidienne et des poids proportionnels à la distance

#### Modele Optimal

In [37]:
# Modèle optimisé
knn_opt = KNeighborsRegressor(n_neighbors=best_params_knn["n_neighbors"],weights=best_params_knn["weights"],p=best_params_knn["p"])
# Entraînement
knn_opt.fit(X_train_scaled, y_train)
# Prédictions
y_pred_opt = knn_opt.predict(X_test_scaled)
# Évaluation
rmse_knn_opt = np.sqrt(mean_squared_error(y_test, y_pred_opt))
r2_knn_opt = r2_score(y_test, y_pred_opt)
print(f"RMSE KNN optimisé : {rmse_knn_opt:.2f}, R2 : {r2_knn_opt:.3f}")

RMSE KNN optimisé : 360.39, R2 : 0.975


### **Random Forest**

#### Modele de référence sans optimisation de paramètres

In [54]:
# Modèle de base
rf = RandomForestRegressor(random_state=42, n_jobs=-1)
# Entraînement
rf.fit(X_train, y_train)
# Prédiction
y_pred_rf = rf.predict(X_test)
# Évaluation
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)
print(f"RMSE RF de base : {rmse_rf:.2f}, R2 RF : {r2_rf:.3f}")

RMSE RF de base : 80.21, R2 RF : 0.999


Le modele de base présente un R2 de 0.99 signe de surapprentissage dû à la non régularisation des arbres, la profondeur et le nombre d'echantillon par feuilles, nous allons donc à la recherche des hyperparamètres

#### Recherche d'hyperparamètres

In [ ]:
# Modèle de base
rf = RandomForestRegressor(random_state=42)
# Grille d’hyperparamètres
param_grid = {'n_estimators': [50, 100, 200,300], 'max_depth': [None, 5,10,15, 20, 30], 'min_samples_split': [2, 5, 10],'min_samples_leaf': [1, 2, 4], 'max_features': ['auto', 'sqrt', 'log2'],'bootstrap': [True, False]}
# GridSearch avec 5-fold cross-validation
grid_search = GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,scoring='neg_root_mean_squared_error', n_jobs=-1,verbose=1)
# Entraînement
grid_search.fit(X_train, y_train)
# Meilleurs paramètres et performances
best_params = grid_search.best_params_
best_rmse = -grid_search.best_score_
print("Meilleurs paramètres RF :", best_params)

Fitting 10 folds for each of 1296 candidates, totalling 12960 fits


/opt/anaconda3/lib/python3.13/multiprocessing/queues.py:120: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/opt/anaconda3/lib/python3.13/multiprocessing/queues.py:120: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/opt/anaconda3/lib/python3.13/multiprocessing/queues.py:120: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPi

Meilleurs paramètres RF : {'bootstrap': False, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
RMSE CV moyen : 178.86


#### Modèle Optimal

In [56]:
# Meilleurs paramètres trouvés
best_params = {'bootstrap': False,'max_depth': 20,'max_features': 'sqrt','min_samples_leaf': 1,'min_samples_split': 2,'n_estimators': 300,'random_state': 42,'n_jobs': -1}
# Modèle optimisé
rf_opt = RandomForestRegressor(**best_params)
# Entraînement
rf_opt.fit(X_train, y_train)
# Prédiction sur le test set
y_pred_rf_opt = rf_opt.predict(X_test)
# Évaluation des performances
rmse_rf_opt = np.sqrt(mean_squared_error(y_test, y_pred_rf_opt))
r2_rf_opt = r2_score(y_test, y_pred_rf_opt)
print(f"RMSE RF optimisé sur test : {rmse_rf_opt:.2f}")
print(f"R2 RF optimisé sur test : {r2_rf_opt:.3f}")

RMSE RF optimisé sur test : 184.83
R2 RF optimisé sur test : 0.993
